In [1]:
import torch
from torch.nn.functional import pad
import torch.nn
from data import YesNoDataset, LibriSpeechDataset, get_dataloader
from model import CausalConformerModel
from torchaudio.functional import rnnt_loss
from torch.nn.functional import log_softmax
from torchmetrics.functional import char_error_rate
from tqdm import tqdm

/home/shibutani/miniconda3/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DEVICE = torch.device("cuda:4" if torch.cuda.is_available() else "cpu")

In [3]:
"""
import json
libri_train_dataset = LibriSpeechDataset(
    root="datasets/",
    split="train",
    vocab_file_path="vocabs/librispeech_train_960h.json",
)
idx_to_audio_sec = {}
for i in range(len(libri_train_dataset)):
    print(f"{i / len(libri_train_dataset) * 100:.2f}%", end="\r")
    audio_sec = libri_train_dataset[i][-1]
    idx_to_audio_sec[i] = audio_sec
with open("librispeech_train_960h_idx_to_audio_sec.json", "w") as f:
    json.dump(idx_to_audio_sec, f)

import json
libri_dev_other_dataset = LibriSpeechDataset(
    root="datasets/",
    split="dev-other",
    vocab_file_path="vocabs/librispeech_dev_other.json",
)
idx_to_audio_sec = {}
for i in range(len(libri_dev_other_dataset)):
    print(f"{i / len(libri_dev_other_dataset) * 100:.2f}%", end="\r")
    audio_sec = libri_dev_other_dataset[i][-1]
    idx_to_audio_sec[i] = audio_sec
with open("librispeech_dev_other_idx_to_audio_sec.json", "w") as f:
    json.dump(idx_to_audio_sec, f)

"""

'\nimport json\nlibri_train_dataset = LibriSpeechDataset(\n    root="datasets/",\n    split="train",\n    vocab_file_path="vocabs/librispeech_train_960h.json",\n)\nidx_to_audio_sec = {}\nfor i in range(len(libri_train_dataset)):\n    print(f"{i / len(libri_train_dataset) * 100:.2f}%", end="\r")\n    audio_sec = libri_train_dataset[i][-1]\n    idx_to_audio_sec[i] = audio_sec\nwith open("librispeech_train_960h_idx_to_audio_sec.json", "w") as f:\n    json.dump(idx_to_audio_sec, f)\n\nimport json\nlibri_dev_other_dataset = LibriSpeechDataset(\n    root="datasets/",\n    split="dev-other",\n    vocab_file_path="vocabs/librispeech_dev_other.json",\n)\nidx_to_audio_sec = {}\nfor i in range(len(libri_dev_other_dataset)):\n    print(f"{i / len(libri_dev_other_dataset) * 100:.2f}%", end="\r")\n    audio_sec = libri_dev_other_dataset[i][-1]\n    idx_to_audio_sec[i] = audio_sec\nwith open("librispeech_dev_other_idx_to_audio_sec.json", "w") as f:\n    json.dump(idx_to_audio_sec, f)\n\n'

In [4]:
"""
import json
yesno_train_dataset = YesNoDataset(
    wav_dir_path="datasets/waves_yesno/",
    model_sample_rate=16000,
    split="train"
)
idx_to_audio_sec = {}
for i in range(len(yesno_train_dataset)):
    print(f"{i / len(yesno_train_dataset) * 100:.2f}%", end="\r")
    audio_sec = yesno_train_dataset[i][-1]
    idx_to_audio_sec[i] = audio_sec
with open("yesno_train_idx_to_audio_sec.json", "w") as f:
    json.dump(idx_to_audio_sec, f)
import json
yesno_dev_dataset = YesNoDataset(
    wav_dir_path="datasets/waves_yesno/",
    model_sample_rate=16000,
    split="dev"
)
idx_to_audio_sec = {}
for i in range(len(yesno_dev_dataset)):
    print(f"{i / len(yesno_dev_dataset) * 100:.2f}%", end="\r")
    audio_sec = yesno_dev_dataset[i][-1]
    idx_to_audio_sec[i] = audio_sec
with open("yesno_dev_idx_to_audio_sec.json", "w") as f:
    json.dump(idx_to_audio_sec, f)
"""

'\nimport json\nyesno_train_dataset = YesNoDataset(\n    wav_dir_path="datasets/waves_yesno/",\n    model_sample_rate=16000,\n    split="train"\n)\nidx_to_audio_sec = {}\nfor i in range(len(yesno_train_dataset)):\n    print(f"{i / len(yesno_train_dataset) * 100:.2f}%", end="\r")\n    audio_sec = yesno_train_dataset[i][-1]\n    idx_to_audio_sec[i] = audio_sec\nwith open("yesno_train_idx_to_audio_sec.json", "w") as f:\n    json.dump(idx_to_audio_sec, f)\nimport json\nyesno_dev_dataset = YesNoDataset(\n    wav_dir_path="datasets/waves_yesno/",\n    model_sample_rate=16000,\n    split="dev"\n)\nidx_to_audio_sec = {}\nfor i in range(len(yesno_dev_dataset)):\n    print(f"{i / len(yesno_dev_dataset) * 100:.2f}%", end="\r")\n    audio_sec = yesno_dev_dataset[i][-1]\n    idx_to_audio_sec[i] = audio_sec\nwith open("yesno_dev_idx_to_audio_sec.json", "w") as f:\n    json.dump(idx_to_audio_sec, f)\n'

In [5]:
dataset = YesNoDataset(
    wav_dir_path="datasets/waves_yesno/",
    model_sample_rate=16000,
    split="train"
)
dataloader = get_dataloader(
    dataset,
    batch_sec=60,
    num_workers=1,
    pad_idx=dataset.pad_idx,
    pin_memory=True,
    audio_sec_file_path="audio_secs/yesno_train_idx_to_audio_sec.json"
)

idx_to_token = dataset.idx_to_token

In [8]:
sampled_idx = []
for bidx, bx, by, bx_len, by_len, baudio_sec in dataloader:
    sampled_idx.append(bidx)
    print(sum(baudio_sec))
print(sampled_idx)
sampled_indices = [item for sublist in sampled_idx for item in sublist]
print(len(set(sampled_indices)) == len(dataset))

Batch Prepare: 100%|██████████| 48/48 [00:00<00:00, 57260.12it/s]


61.89999999999999
61.089999999999996
62.24
61.489999999999995
47.690000000000005
[[45, 27, 17, 40, 10, 42, 9, 5, 6, 46], [18, 32, 2, 44, 13, 8, 30, 0, 15, 28], [19, 21, 22, 26, 41, 36, 29, 33, 47, 14], [12, 20, 11, 25, 38, 34, 24, 31, 16, 37], [7, 3, 1, 43, 23, 35, 4, 39]]
True


In [10]:
libri_dataset = LibriSpeechDataset(
    root="./datasets",
    split="dev-other",
    resampling_rate=16000,
    vocab_file_path="vocabs/librispeech_train_960h.json",
    audio_sec_file_path="audio_secs/librispeech_dev_other_idx_to_audio_sec.json",
)
libri_dataloader = get_dataloader(
    libri_dataset,
    batch_sec=100,
    num_workers=1,
    pad_idx=libri_dataset.pad_idx,
    pin_memory=True
)

In [11]:
sampled_idx = []
for bidx, bx, by, bx_len, by_len, baudio_sec in libri_dataloader:
    sampled_idx.append(bidx)
    print(sum(baudio_sec))
print(sampled_idx)
sampled_indices = [item for sublist in sampled_idx for item in sublist]
print(len(set(sampled_indices)) == len(libri_dataset))

Batch Prepare: 100%|██████████| 2864/2864 [00:00<00:00, 227165.03it/s]


104.3
108.2000625
103.50493749999998
103.26500000000001
101.39
100.86000000000001
102.2900625
100.165
110.26000000000002
102.50499999999998
106.365
113.99493750000002
105.58
100.73000000000003
103.325
100.08506249999999
102.78
103.66
101.485
101.24000000000001
103.45500000000001
100.51
113.58999999999997
110.14500000000001
100.25500000000001
104.57
103.60499999999999
101.91
104.29
106.86500000000001
120.35
104.61999999999999
100.48500000000003
112.49506249999999
108.15499999999999
100.465
107.035
107.25493750000001
107.21993749999997
101.815
109.61500000000001
100.24000000000001
105.28500000000001
111.63499999999999
101.805
101.22499999999998
100.03993749999998
109.87493750000002
106.05000000000001
102.0700625
109.9050625
103.905
100.59
106.32993749999997
102.845
112.18506249999999
101.06500000000001
102.505
105.1250625
105.08
100.13499999999999
102.13499999999999
105.44500000000001
102.36500000000001
100.9399375
104.895
106.13
107.345
109.03500000000001
102.57006249999999
102.26493749

In [46]:
cpt_path = "./cpts/yesno_causal_conformer/836b576aac7744c2857c00ae170f7b91/model_YesNo_80.pth"
with open(cpt_path, "rb") as f:
    cpt = torch.load(f, map_location=DEVICE)

model = CausalConformerModel(
    vocab_size=8,
    encoder_input_size=40,
    encoder_subsampled_input_size=128,
    encoder_num_conformer_blocks=1,
    encoder_ff_hidden_size=128,
    encoder_conv_hidden_size=128,
    encoder_conv_kernel_size=16,
    encoder_mha_num_heads=4,
    encoder_dropout=0,
    encoder_subsampling_kernel_size1=3,
    encoder_subsampling_stride1=2,
    encoder_subsampling_kernel_size2=3,
    encoder_subsampling_stride2=2,
    encoder_num_previous_frames="all",
    embedding_size=64,
    predictor_hidden_size=64,
    predictor_num_layers=1,
    jointnet_hidden_size=64,
    blank_idx=dataset.blank_idx,
    decoder_buffer_size=4,
).to(DEVICE)
model.load_state_dict(cpt["model"])


<All keys matched successfully>

In [48]:
model.eval()
benc_input, bpred_input, benc_input_length, bpred_input_length, baudio_sec = next(iter(dataloader))
benc_input = benc_input.to(DEVICE)
bhyp_token_indices = model.streaming_greedy_inference(benc_input, benc_input_length)
bhyp_token_indices

[[0,
  1,
  2,
  5,
  3,
  4,
  5,
  3,
  4,
  5,
  3,
  4,
  5,
  3,
  4,
  5,
  3,
  4,
  5,
  0,
  1,
  2,
  5,
  0,
  1,
  2],
 [0,
  1,
  2,
  5,
  0,
  1,
  2,
  5,
  3,
  4,
  5,
  3,
  4,
  5,
  0,
  1,
  2,
  5,
  3,
  4,
  5,
  0,
  1,
  2,
  5,
  3,
  4],
 [0,
  1,
  2,
  5,
  3,
  4,
  5,
  0,
  1,
  2,
  5,
  0,
  1,
  2,
  5,
  0,
  1,
  2,
  5,
  0,
  1,
  2,
  5,
  3,
  4,
  5,
  0,
  1,
  2],
 [0,
  1,
  2,
  5,
  0,
  1,
  2,
  5,
  0,
  1,
  2,
  5,
  0,
  1,
  2,
  5,
  3,
  4,
  5,
  0,
  1,
  2,
  5,
  3,
  4,
  5,
  3,
  4],
 [3,
  4,
  5,
  3,
  4,
  5,
  0,
  1,
  2,
  5,
  0,
  1,
  2,
  5,
  0,
  1,
  2,
  5,
  3,
  4,
  5,
  3,
  4,
  5,
  3,
  4],
 [3,
  4,
  5,
  0,
  1,
  2,
  5,
  0,
  1,
  2,
  5,
  0,
  1,
  2,
  5,
  0,
  1,
  2,
  5,
  0,
  1,
  2,
  5,
  0,
  1,
  2,
  5,
  0,
  1,
  2],
 [3,
  4,
  5,
  0,
  1,
  2,
  5,
  3,
  4,
  5,
  0,
  1,
  2,
  5,
  0,
  1,
  2,
  5,
  0,
  1,
  2,
  5,
  3,
  4,
  5,
  3,
  4],
 [0,
  1,
  2,
  5,
  3,
  4

In [49]:
bans_token_indices = [
    bpred_input[i, : bpred_input_length[i]].tolist() for i in range(bpred_input.shape[0])
]
bhyp_text = [
    "".join([idx_to_token[idx] for idx in hyp_token_indices])
    for hyp_token_indices in bhyp_token_indices
]
bans_text = [
    "".join([idx_to_token[idx] for idx in ans_token_indices])
    for ans_token_indices in bans_token_indices
]

In [50]:
for i in range(len(bhyp_text)):
    print(f"hyp: {bhyp_text[i]}")
    print(f"ans: {bans_text[i]}")
    print()

hyp: yes no no no no no yes yes
ans: yes no no no no no yes yes

hyp: yes yes no no yes no yes no
ans: yes yes no no yes no yes no

hyp: yes no yes yes yes yes no yes
ans: yes no yes yes yes yes no yes

hyp: yes yes yes yes no yes no no
ans: yes yes yes yes no yes no no

hyp: no no yes yes yes no no no
ans: no no yes yes yes no no no

hyp: no yes yes yes yes yes yes yes
ans: no yes yes yes yes yes yes yes

hyp: no yes no yes yes yes no no
ans: no yes no yes yes yes no no

hyp: yes no yes yes yes no yes no
ans: yes no yes yes yes no yes no



In [13]:
x = torch.ones(3, 10, 5)
input_length=x.shape[1]
future_mask = torch.triu(torch.ones(input_length, input_length), diagonal=1).bool()
print(future_mask)
NUM_PREVIOUS_FRAMES = "all"
# mask before NUM_PREVIOUS_FRAMES
input_length=x.shape[1]
if NUM_PREVIOUS_FRAMES == "all":
    previous_mask = torch.zeros(input_length, input_length).bool()
else:
    previous_mask=torch.tril(torch.ones(input_length, input_length), diagonal=-(NUM_PREVIOUS_FRAMES+1)).bool()
print(previous_mask)
future_and_previous_mask = torch.logical_or(future_mask, previous_mask)

tensor([[False,  True,  True,  True,  True,  True,  True,  True,  True,  True],
        [False, False,  True,  True,  True,  True,  True,  True,  True,  True],
        [False, False, False,  True,  True,  True,  True,  True,  True,  True],
        [False, False, False, False,  True,  True,  True,  True,  True,  True],
        [False, False, False, False, False,  True,  True,  True,  True,  True],
        [False, False, False, False, False, False,  True,  True,  True,  True],
        [False, False, False, False, False, False, False,  True,  True,  True],
        [False, False, False, False, False, False, False, False,  True,  True],
        [False, False, False, False, False, False, False, False, False,  True],
        [False, False, False, False, False, False, False, False, False, False]])
tensor([[False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False, Fa

In [15]:
future_and_previous_mask

tensor([[False,  True,  True,  True,  True,  True,  True,  True,  True,  True],
        [False, False,  True,  True,  True,  True,  True,  True,  True,  True],
        [False, False, False,  True,  True,  True,  True,  True,  True,  True],
        [False, False, False, False,  True,  True,  True,  True,  True,  True],
        [False, False, False, False, False,  True,  True,  True,  True,  True],
        [False, False, False, False, False, False,  True,  True,  True,  True],
        [False, False, False, False, False, False, False,  True,  True,  True],
        [False, False, False, False, False, False, False, False,  True,  True],
        [False, False, False, False, False, False, False, False, False,  True],
        [False, False, False, False, False, False, False, False, False, False]])